# COG vs Zarr

COG vs zarr is a big point of contention in the community right now. Let's look at raster data and how it is stored in the Cloud-Optimized GeoTIFF (COG) format, and how that compares to what zarr does with the same data.

Before we get into this we need to get some imports out of the way.

In [41]:
import json
import shutil
import struct
import urllib.request
import zlib

from hashlib import sha256
from pathlib import Path

import numpy as np
import rasterio
import rioxarray
import xarray

from crc32c import crc32c
from numcodecs.zarr3 import Zlib, Delta
from numcodecs import Zstd
from rio_cogeo.cogeo import cog_translate, cog_validate
from rio_cogeo.profiles import cog_profiles
from tifffile import TiffFile

And we will want to be comparing some byte strings. An easy way to identify a sequence of bytes is to use a SHASUM hash, so let's make a little function to generate a hash in hex format and display that along with the on-disk size of the byte string. We'll also be be displaying some dictonaries and JSON files, so let's make a couple little helper functions to display those in a pretty manner.

In [2]:
def describe_bytes(b: bytes) -> str:
    h = sha256()
    h.update(b)
    print(f"size: {len(b) / 2**20:.3f} MiB | shasum: {h.digest().hex()}")


def print_dict(d: dict) -> None:
    print(json.dumps(d, indent=4))


def JSON(p: Path) -> None:
    print_dict(json.loads(p.read_text()))

We also have to do a bit of custom data munging to support some compression-related things that are a little different between zarr and COG. The following code cell can be ignored for the moment; come back to it in the discussion around predictors, but only if you are curious.

In [4]:
from numcodecs import register_codec as _register_codec
import numcodecs.delta as _delta
import numcodecs.zarr3 as _z3
from zarr.registry import register_codec as _register_codec_v3

# from https://github.com/zarr-developers/numcodecs/issues/583
# tiff uses a delta along a specific dimension, so we need this
# codec for compatibility with the tiff predictor=2
class SpatialDelta(_delta.Codec):
    codec_id = 'spatial_delta'

    def __init__(self, axis, dtype, astype=None):
        self.axis = axis
        self.dtype = np.dtype(dtype)
        if astype is None:
            self.astype = self.dtype
        else:
            self.astype = np.dtype(astype)
        if self.dtype == np.dtype(object) or self.astype == np.dtype(object):
            raise ValueError('object arrays are not supported')

    def encode(self, buf):
        # normalise input
        arr = _delta.ensure_ndarray(buf).view(self.dtype)

        # flatten to simplify implementation
        # arr = arr.reshape(-1, order='A')

        # setup encoded output
        enc = np.empty_like(arr, dtype = self.astype)

        # set first element
        slice_idx = [slice(0, None) for _ in range(arr.ndim)]
        slice_idx[self.axis] = slice(0,1)
        enc[*slice_idx] = arr[*slice_idx]

        # compute differences
        slice_idx[self.axis] = slice(1, None)
        enc[*slice_idx] = np.diff(arr, axis = self.axis)

        return enc

    def decode(self, buf, out=None):
        # normalise input
        enc = _delta.ensure_ndarray(buf).view(self.astype)

        # flatten to simplify implementation

        # setup decoded output
        dec = np.empty_like(enc, dtype=self.dtype)

        # decode differences
        np.cumsum(enc, axis = self.axis, out=dec)

        # handle output
        out = _delta.ndarray_copy(dec, out)

        return out

    def get_config(self):
        # override to handle encoding dtypes
        return dict(id=self.codec_id, dtype=self.dtype.str, astype=self.astype.str)

    def __repr__(self):
        r = f'{type(self).__name__}(dtype={self.dtype.str!r}, axis={self.axis}'
        if self.astype != self.dtype:
            r += f', astype={self.astype.str!r}'
        r += ')'
        return r


_register_codec(SpatialDelta)


class SpatialDelta3(_z3._NumcodecsArrayArrayCodec):
    codec_name = f"numcodecs.spatial_delta"

    def __init__(self, **codec_config: dict[str, _z3.JSON]) -> None:
        super().__init__(**codec_config)

    def resolve_metadata(self, chunk_spec: _z3.ArraySpec) -> _z3.ArraySpec:
        if astype := self.codec_config.get("astype"):
            return _z3.replace(chunk_spec, dtype=np.dtype(astype))  # type: ignore[call-overload]
        return chunk_spec


_register_codec_v3(SpatialDelta3.codec_name, SpatialDelta3)

To organize our data we'll use a specific directory:

In [5]:
OUTDIR = Path('test_data')
OUTDIR.mkdir(exist_ok=True)

## Downloading a COG

We're going to use a Sentinel 2 L2A scene for this exploration. To keep things simple we'll just use one band, and we can link directly to a COG. Specifically, we'll use the red band of the scene `S2B_T10TFR_20231223T190950_L2A`. Let's download that file now.

<div class="alert alert-block alert-info">
<b>Note:</b> This is a simplified workflow. In practice you might get this href by searching for the scene of interest using the <a href="https://earth-search.aws.element84.com/v1">Earth Search STAC API</a>.
</div>

In [6]:
COG_HREF = 'https://e84-earth-search-sentinel-data.s3.us-west-2.amazonaws.com/sentinel-2-c1-l2a/10/T/FR/2023/12/S2B_T10TFR_20231223T190950_L2A/B04.tif'
COG_FILE = OUTDIR / 'red.tif'

# check if we are rerunning this cell to not download the COG if we already have it
if not COG_FILE.exists():
    with urllib.request.urlopen(urllib.request.Request(COG_HREF)) as response:
        COG_FILE.write_bytes(response.read())

## Reading the COG metadata

Now that we have the COG, we can open it with `tifffile` and read the tags out to see what kind of metadata the file has. Note here that the TIFF file has mutliple "pages" in it, or "images". This is because the COG includes overviews, which internally in the TIFF are stored as what TIFF terms individual images. `tifffile` uses the term "pages" instead of images; the first page here is the full resolution data, and each successive page is the next overview up from the last.

We'll store the tags in a dictionary so we can use the values later as needed.

In [7]:
cog_tags = {}
with TiffFile(COG_FILE) as tif:
    for tag in tif.pages[0].tags:
        cog_tags[tag.name] = tag.value

print_dict(cog_tags)

{
    "ImageWidth": 10980,
    "ImageLength": 10980,
    "BitsPerSample": 16,
    "Compression": 8,
    "PhotometricInterpretation": 1,
    "SamplesPerPixel": 1,
    "PlanarConfiguration": 1,
    "Predictor": 2,
    "TileWidth": 1024,
    "TileLength": 1024,
    "TileOffsets": [
        55962680,
        57411167,
        58810332,
        60222446,
        61651003,
        63054996,
        64463518,
        66025043,
        67523672,
        68987825,
        70439668,
        71480485,
        72831139,
        74191906,
        75556803,
        76922917,
        78346396,
        79767466,
        81177106,
        82626646,
        84045343,
        85436959,
        86443457,
        87744763,
        89128625,
        90516041,
        91896145,
        93323921,
        94699513,
        96054131,
        97398784,
        98768288,
        100117176,
        101099165,
        102475360,
        103914015,
        105337327,
        106767167,
        108155055,
        109

## Extracting a tile

The tags contain a bunch of great stuff we could discuss at length, but the first thing I think it is worth pointing out is that the tiles in this COG are stored at various offsets within the file. We can see the `TileOffsets` tag contains a sequence of those offsets. Similarly, `TileByteCounts` contains a sequence of the lengths of each of those tiles. Thus, we can read tile (0,0) by grabbing the first value in each of those arrays and using them to seek to and read the tile's bytes. We can hash those bytes to get a unique identifer for those bytes we can print out.

In [9]:
with (COG_FILE).open('rb') as tif:
    tif.seek(cog_tags['TileOffsets'][0])
    cog_tile_bytes = tif.read(cog_tags['TileByteCounts'][0])

describe_bytes(cog_tile_bytes)

size: 1.381 MiB | shasum: 2c02e7e60074d6767ccb4c44de2da249d331fd82e107431e41cfe4069bae0d62


## Creating a zarr from the COG

The library `rio-xarray` contains a number of conveniences for working with geospatial raster data with `xarray`, and `xarray` can write zarrs. So if we use `rio-xarray` to open our COG as an `xarray.Dataset`, we can transform our COG into a zarr. First things first though, let's start by simply opening the file as an `xarray.Dataset`.

In [10]:
ds = rioxarray.open_rasterio(COG_FILE, band_as_variable=True).rename_vars(band_1="red")
ds

<xarray.Dataset> Size: 241MB
Dimensions:      (x: 10980, y: 10980)
Coordinates:
  * x            (x) float64 88kB 6e+05 6e+05 6e+05 ... 7.098e+05 7.098e+05
  * y            (y) float64 88kB 5.1e+06 5.1e+06 5.1e+06 ... 4.99e+06 4.99e+06
    spatial_ref  int64 8B 0
Data variables:
    red          (y, x) uint16 241MB ...
Attributes:
    OVR_RESAMPLING_ALG:  AVERAGE
    AREA_OR_POINT:       Area

Notice that we have the same dimensions as we saw in the COG metadata! That's a good sign. Now let's try writing out our data as zarr v3 using the default settings. We can open it back up to see it is in fact a zarr, too.

In [11]:
ZARR_DEFAULTS = OUTDIR / 'zarr_defaults'

# just in case we've run this before
if ZARR_DEFAULTS.exists():
    shutil.rmtree(ZARR_DEFAULTS)

ds.to_zarr(
    ZARR_DEFAULTS,
    zarr_format=3,
)
xarray.open_dataset(ZARR_DEFAULTS, engine='zarr')

/Users/jkeifer/dev/cng-raster-formats/.direnv/python-3.12/lib/python3.12/site-packages/zarr/api/asynchronous.py:205: UserWarning: Consolidated metadata is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  warnings.warn(


<xarray.Dataset> Size: 965MB
Dimensions:      (y: 10980, x: 10980)
Coordinates:
  * x            (x) float64 88kB 6e+05 6e+05 6e+05 ... 7.098e+05 7.098e+05
  * y            (y) float64 88kB 5.1e+06 5.1e+06 5.1e+06 ... 4.99e+06 4.99e+06
Data variables:
    red          (y, x) float64 964MB ...
    spatial_ref  int64 8B ...
Attributes:
    OVR_RESAMPLING_ALG:  AVERAGE
    AREA_OR_POINT:       Area

### Reading the zarr metadata

The metadata for zarr is a bit of a contentious topic due to consolidated metadata vs not--we're going to sidestep an opinion in that discussion for the moment and just say that our relevant metadata in this v3 store is stored in the file `red/zarr.json`. Let's open that up and dump its contents to compare with our TIFF tag metadata.

In [12]:
JSON(ZARR_DEFAULTS / 'red' / 'zarr.json')

{
    "shape": [
        10980,
        10980
    ],
    "data_type": "uint16",
    "chunk_grid": {
        "name": "regular",
        "configuration": {
            "chunk_shape": [
                687,
                1373
            ]
        }
    },
    "chunk_key_encoding": {
        "name": "default",
        "configuration": {
            "separator": "/"
        }
    },
    "fill_value": 0,
    "codecs": [
        {
            "name": "bytes",
            "configuration": {
                "endian": "little"
            }
        },
        {
            "name": "zstd",
            "configuration": {
                "level": 0,
                "checksum": false
            }
        }
    ],
    "attributes": {
        "STATISTICS_MAXIMUM": 17408,
        "STATISTICS_MEAN": 1505.1947339533,
        "STATISTICS_MINIMUM": 294,
        "STATISTICS_STDDEV": 659.24503616433,
        "STATISTICS_VALID_PERCENT": 99.999,
        "scale_factor": 0.0001,
        "add_offset": -0.1,
 

Wow, for the most part that looks kinda similar. It is missing spatial reference information directly, which is an important note. But probably the first thing that looks like a problem is the chunk size.

Zarr stores data very similarly to COG in that it spilts the data up into smaller pieces. Zarr terms these "chunks", as opposed to the COG nomenclature of "tiles", but they are functionally equivalent. Except here we see the chunk size used is 687 x 1373, not 1024 x 1024. To ensure we can effectively compare the data in a zarr chunk to one of our COG tiles we should make these match in size.

## Writing zarr with a specific chunk size

Turns out we can use the `encoding` argument of `to_zarr` to specify the chunk size we want to use. We have to do that on a per-data-variable-basis, so we have to nest our encoding settings within a dictionary keyed off the name of our data variable, which we previously specified was `red`.

In [25]:
ZARR_TILED = OUTDIR / 'zarr_tiled'

# just in case we've run this before
if ZARR_TILED.exists():
    shutil.rmtree(ZARR_TILED)

ds.to_zarr(
    ZARR_TILED,
    zarr_format=3,
    encoding={
        'red': {
            "chunks": (1024, 1024),
        },
    },
)

/Users/jkeifer/dev/cng-raster-formats/.direnv/python-3.12/lib/python3.12/site-packages/zarr/api/asynchronous.py:205: UserWarning: Consolidated metadata is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  warnings.warn(


The metadata is in the same file within this new zarr store, so we can read it out the same as before to see if that looks any better.

In [26]:
JSON(ZARR_TILED / 'red' / 'zarr.json')

{
    "shape": [
        10980,
        10980
    ],
    "data_type": "uint16",
    "chunk_grid": {
        "name": "regular",
        "configuration": {
            "chunk_shape": [
                1024,
                1024
            ]
        }
    },
    "chunk_key_encoding": {
        "name": "default",
        "configuration": {
            "separator": "/"
        }
    },
    "fill_value": 0,
    "codecs": [
        {
            "name": "bytes",
            "configuration": {
                "endian": "little"
            }
        },
        {
            "name": "zstd",
            "configuration": {
                "level": 0,
                "checksum": false
            }
        }
    ],
    "attributes": {
        "STATISTICS_MAXIMUM": 17408,
        "STATISTICS_MEAN": 1505.1947339533,
        "STATISTICS_MINIMUM": 294,
        "STATISTICS_STDDEV": 659.24503616433,
        "STATISTICS_VALID_PERCENT": 99.999,
        "scale_factor": 0.0001,
        "add_offset": -0.1,


Indeed it does! Now that we have the same tile/chunk sizing between our COG and our zarr, how do the bytes compare between the two?

### Reading a zarr chunk

Unlike COG, where everything is stored within a single file, by default zarr uses a separate file for metadata, as we have seen, as well as a separate file per chunk. Navigate through the zarr directory tree, and notice that we have a directory tree like `red/c/0/[0..10]/[0..10]`, as we have 1 chunk in the band dimension, and 11 chunks in each spatial dimension x and y. This breaks down such that the data for our tile (0,0) is located at the path `red/c/0/0/0` within the store. We can open that up, read its bytes, and find its length and hash to compare to our COG tile (0,0).

In [27]:
zarr_tile_bytes = (ZARR_TILED / 'red' / 'c' / '0' / '0').read_bytes()

describe_bytes(cog_tile_bytes)
describe_bytes(zarr_tile_bytes)

size: 1.381 MiB | shasum: 2c02e7e60074d6767ccb4c44de2da249d331fd82e107431e41cfe4069bae0d62
size: 1.387 MiB | shasum: 6bad8a3594bbdf9300c7f823a5969ece06d6f596d9139908c2f01de51e564af8


Hmm, those are close lengths, but the bytes are not the same. Should they be?

## Compression codecs

Turns out, maybe! But to understand better why that could maybe be true we need to look more at the COG and zarr metadata to see what other differences we can spot beyond the original tile/chunk size difference.

Notice in the tiff tags we see `Compression` has the value 8. Similarly, we see in the zarr metadata under the `codecs` that `zstd` is specified. Both of these bits of metadata are indicating how the tiles/chunks are compressed. In the case of the COG, the value of the `Compression` tag requires we have an external lookup table to interpret. [Wikipedia has a great lookup table](https://en.wikipedia.org/wiki/TIFF#TIFF_Compression_Tag) we can use to see what this value of 8 indicates--turns out it means the tiles are each individually compressed using the `DEFLATE` algorithm.

On the zarr side, the metadata is more verbose and is self documenting: `zstd` is a different compression algorithm. So we see that using the zarr defaults we did not end up using the same compression that our COG uses. If we correct that mismatch will our tile bytes be the same?

### Predictors

Before we waste too much time trying out `DEFLATE` on our zarr just to find another difference, let me shortcut this process by pointing out one more difference. Our COG metadata indicates a `Predictor` of 2. What is this?

Predictors are filters that can be run on the data prior to compression to increase the compressibility of the data. The TIFF specification has support for a few different predictors, but the main three values are as follows:

* 1: no predictor used prior to compression
* 2: delta predictor; this calculates the horizontal difference between cells in each tile row, only useful for integer data
* 3: [floating point byte reordering](http://chriscox.org/TIFFTN3d1.pdf), only useful for floating point data

Zarr also supports predictors, but does so via the more flexible paradigm of "filters". Filters can be used for any data transformations that need to happen prior to compressing data, or be reversed after decompressing data. Predictors are just one type of a filter; another example could be like scaling, offsetting, and casting values to allow transforming floats and/or signed values into smaller, less complicated unsigned integers (and in fact our COG has scaled/offset values, with no standard way (as in, within the TIFF specification) to apply those transformations).

So putting all this together, we need change our zarr configuration to use `DEFLATE` compression and a predictor to see if we can match our COG data. Let's try it out!

<div class="alert alert-block alert-warning">
<b>Be aware:</b> numcodecs includes a Delta codec that seems like ti would be fit for the purpose of fulfilling the role of predictor 2. Except it reshapes the array to a single dimension then runs the difference over the whole thing, which is not compatible with the TIFF delta that only calcualtes the horizontal difference within a given row. So we are using a custom SpatialDelta3 codec here to ensure compatibility with TIFF and how we have stored the data in the TIFF.
</div>

In [28]:
ZARR_DEFLATE = OUTDIR / 'zarr_deflate'

# just in case we've run this before
if ZARR_DEFLATE.exists():
    shutil.rmtree(ZARR_DEFLATE)

ds.to_zarr(
    ZARR_DEFLATE,
    zarr_format=3,
    encoding={
        'red': {
            # axis=1 is will calculate the difference along our array rows
            'filters': [SpatialDelta3(axis=1, dtype='uint16', astype='uint16')],
            # we just happen to know the compression level on Earth Search is max
            'compressors': [Zlib(level=9)],
            'chunks': (1024, 1024),
        },
    },
)

/Users/jkeifer/dev/cng-raster-formats/.direnv/python-3.12/lib/python3.12/site-packages/numcodecs/zarr3.py:155: UserWarning: Numcodecs codecs are not in the Zarr version 3 specification and may not be supported by other zarr implementations.
  super().__init__(**codec_config)
/Users/jkeifer/dev/cng-raster-formats/.direnv/python-3.12/lib/python3.12/site-packages/numcodecs/zarr3.py:133: UserWarning: Numcodecs codecs are not in the Zarr version 3 specification and may not be supported by other zarr implementations.
  super().__init__(**codec_config)


/Users/jkeifer/dev/cng-raster-formats/.direnv/python-3.12/lib/python3.12/site-packages/numcodecs/zarr3.py:155: UserWarning: Numcodecs codecs are not in the Zarr version 3 specification and may not be supported by other zarr implementations.
  super().__init__(**codec_config)
/Users/jkeifer/dev/cng-raster-formats/.direnv/python-3.12/lib/python3.12/site-packages/numcodecs/zarr3.py:133: UserWarning: Numcodecs codecs are not in the Zarr version 3 specification and may not be supported by other zarr implementations.
  super().__init__(**codec_config)
/Users/jkeifer/dev/cng-raster-formats/.direnv/python-3.12/lib/python3.12/site-packages/zarr/api/asynchronous.py:205: UserWarning: Consolidated metadata is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  warnings.warn(


Cool! Let's see how our new chunk bytes compare to our COG.

In [29]:
zarr_deflate_tile_bytes = (ZARR_DEFLATE / 'red' / 'c' / '0' / '0').read_bytes()

describe_bytes(cog_tile_bytes)
describe_bytes(zarr_deflate_tile_bytes)

size: 1.381 MiB | shasum: 2c02e7e60074d6767ccb4c44de2da249d331fd82e107431e41cfe4069bae0d62
size: 1.381 MiB | shasum: 2c02e7e60074d6767ccb4c44de2da249d331fd82e107431e41cfe4069bae0d62


Wow, look at that! The are the same bytes on disk, proving that the bytes representing the data stored in a zarr chunk are _exactly the same_ as the bytes in the portion of a COG file storing the same data, when controling for variables like compression and other filters.

## What does this mean?

Wait, if this is the same data...

Zarr v3 introduced a concept called "shards". The idea with shards is to take some number of an array's chunks and write them into a single file on disk. Why? Shards have some distinct advantages at scale, such as reducing the number of files that need to be managed within the store, and by allowing for more efficient reads when needing consecutive chunks.

But if shards are just chunks put into the same file, what is the difference between a COG and a shard, aside from the COG storing its metadata in the file as well? Turns out, maybe nothing.

### We can build a COG that is a zarr that is a COG!

We're going to need to do some hacking. First, let's build a metadata config for a sharded zarr array for our image data, where all the chunks are stored in a single shard.

In [36]:
DEFLATE_SHARDED_ZARR_CONF = {
    "shape": [
        10980,
        10980
    ],
    "data_type": "uint16",
    "chunk_grid": {
        "name": "regular",
        "configuration": {
            "chunk_shape": [
                11264,
                11264
            ]
        }
    },
    "chunk_key_encoding": {
        "name": "default",
        "configuration": {
            "separator": "/"
        }
    },
    "fill_value": 0,
    "codecs": [
        {
            "name": "sharding_indexed",
            "configuration": {
                "chunk_shape": [
                    1024,
                    1024
                ],
                "codecs": [
                    {
                        "name": "numcodecs.spatial_delta",
                        "configuration": {
                            "axis": 1,
                            "dtype": "uint16",
                            "astype": "uint16"
                        }
                    },
                    {
                        "name": "bytes",
                        "configuration": {
                            "endian": "little"
                        }
                    },
                    {
                        "name": "numcodecs.zlib",
                        "configuration": {
                            "level": 9
                        }
                    }
                ],
                "index_codecs": [
                    {
                        "name": "bytes",
                        "configuration": {
                            "endian": "little"
                        }
                    },
                    {
                        "name": "crc32c"
                    }
                ],
                "index_location": "end"
            }
        }
    ],
    "attributes": {
        "OVR_RESAMPLING_ALG": "AVERAGE",
        "AREA_OR_POINT": "Area",
        "STATISTICS_MAXIMUM": 17408,
        "STATISTICS_MEAN": 1505.1947339533,
        "STATISTICS_MINIMUM": 294,
        "STATISTICS_STDDEV": 659.24503616433,
        "STATISTICS_VALID_PERCENT": 99.999,
        "scale_factor": 0.0001,
        "add_offset": -0.1,
        "_FillValue": 0
    },
    "dimension_names": [
        "y",
        "x"
    ],
    "zarr_format": 3,
    "node_type": "array",
    "storage_transformers": []
}

Now, we're going to do some frankenstein operations, starting by copying our DEFLATE-compressed zarr to a new directory tree--so creatively called `zog`. Then we are going to remove all our `red` data chunks, and instead copy in our COG file as the shard at the path of `red/c/0/0`. With all this in place we'll overwrite our `red` metadata, both in its `zarr.json` and in the consolidated metadata in the top-level `zarr.json`.

After all that we're still not done. See, zarr expects to find a shard index within the shard. Looking at the metadata we constructed above, we see the `index_location` is at the end of the shard file. So we need to construct that index and write it to the end of the file.

The format of that index is rather interesting. It is itself an array, with the dimensions of our shard plus an extra dimension on the end. That is, our shard is shape (11, 11), and our index must be (11, 11, 2). This is because we need two values per chunk in the shard, the byte offset of the start of the chunk and its length in bytes. Just like we already have in our COG tags, just paired up and stored in a multidimensional array.

The array we can cast to bytes so we can append it to the end of our COG. Except we have to do one more thing. Notice in the `index_codecs` in the metadata above we have a codec listed as `crc32c`. This is actually optional, and we could have omitted it here, but it is best practice to include it, so for completeness we will include it. This is a checksum of the index, and helps us verify its integrity. We can calculate it on the bytes from our array; the output is a 32-bit unsigned int, so we can use `struct.pack` to take the Python int value and encode it as a 32-bit unsigned int in bytes, and append that to our index bytes.

Now our index bytes are ready, so we'll write them to the end of our COG.

In [37]:
ZOG = OUTDIR / 'zog'
COG_SHARD = (ZOG / 'red' /  'c' / '0' / '0')

# just in case we've run this before
if ZOG.exists():
    shutil.rmtree(ZOG)

shutil.copytree(ZARR_DEFLATE, ZOG)
shutil.rmtree(COG_SHARD.parent)
COG_SHARD.parent.mkdir()
shutil.copy(COG_FILE, COG_SHARD)
(ZOG / 'red' / 'zarr.json').write_text(json.dumps(DEFLATE_SHARDED_ZARR_CONF, indent=4))
consolidated = json.loads((ZOG / 'zarr.json').read_text())
consolidated['consolidated_metadata']['metadata']['red'] = DEFLATE_SHARDED_ZARR_CONF
(ZOG / 'zarr.json').write_text(json.dumps(consolidated, indent=4))

index_bytes = np.array(list(zip(cog_tags['TileOffsets'], cog_tags['TileByteCounts']))).reshape((11, 11, 2)).tobytes()
index_bytes += struct.pack('I', crc32c(index_bytes))

with COG_SHARD.open('ab') as fh:
    fh.write(index_bytes)

Phew, that was a lot. Let's try this out and see what we get if we read a slice of values from the file. We'll compare this to the original zarr we created back at the beginning to verify the values look correct. To ensure we have our index correct beyond the first tile, let's take a slice on a tile corner to have to load four chunks.

In [38]:
ds_zog = xarray.open_dataset(ZOG, engine='zarr')
ds_zog['red'][1020:1030,1020:1030].values

/Users/jkeifer/dev/cng-raster-formats/.direnv/python-3.12/lib/python3.12/site-packages/numcodecs/zarr3.py:155: UserWarning: Numcodecs codecs are not in the Zarr version 3 specification and may not be supported by other zarr implementations.
  super().__init__(**codec_config)
/Users/jkeifer/dev/cng-raster-formats/.direnv/python-3.12/lib/python3.12/site-packages/numcodecs/zarr3.py:133: UserWarning: Numcodecs codecs are not in the Zarr version 3 specification and may not be supported by other zarr implementations.
  super().__init__(**codec_config)


array([[-0.0029,  0.0003, -0.0015, -0.0017, -0.0003, -0.0053, -0.007 ,
        -0.0085, -0.0038, -0.0017],
       [ 0.0026,  0.0019, -0.0013, -0.0041, -0.0043, -0.0066, -0.0073,
        -0.0081, -0.0037, -0.0016],
       [ 0.0036, -0.0009, -0.0037, -0.006 , -0.007 , -0.0043, -0.0018,
        -0.0039, -0.0051, -0.0004],
       [-0.0042, -0.0041, -0.0055, -0.0053, -0.0088, -0.0071, -0.0021,
        -0.0034, -0.003 ,  0.0005],
       [ 0.0025,  0.0026, -0.0013,  0.0004, -0.0077, -0.0045, -0.0018,
        -0.0039, -0.0023,  0.0009],
       [-0.0056, -0.0044, -0.0044, -0.0044, -0.0028, -0.0013, -0.0023,
        -0.0007,  0.0027,  0.0005],
       [-0.013 , -0.0137, -0.0072, -0.0032, -0.0018, -0.0022, -0.003 ,
         0.0001,  0.0031, -0.0028],
       [-0.0108, -0.0101, -0.0144, -0.009 , -0.0042, -0.0016, -0.0058,
        -0.0079, -0.0035, -0.0069],
       [-0.0078, -0.0051, -0.0088, -0.0113, -0.0075, -0.003 , -0.0046,
        -0.0028, -0.0068, -0.0051],
       [-0.0083, -0.0041, -0.0047, -0

In [39]:
ds_zarr = xarray.open_dataset(ZARR_DEFAULTS, engine='zarr')
ds_zarr['red'][1020:1030,1020:1030].values

array([[-0.0029,  0.0003, -0.0015, -0.0017, -0.0003, -0.0053, -0.007 ,
        -0.0085, -0.0038, -0.0017],
       [ 0.0026,  0.0019, -0.0013, -0.0041, -0.0043, -0.0066, -0.0073,
        -0.0081, -0.0037, -0.0016],
       [ 0.0036, -0.0009, -0.0037, -0.006 , -0.007 , -0.0043, -0.0018,
        -0.0039, -0.0051, -0.0004],
       [-0.0042, -0.0041, -0.0055, -0.0053, -0.0088, -0.0071, -0.0021,
        -0.0034, -0.003 ,  0.0005],
       [ 0.0025,  0.0026, -0.0013,  0.0004, -0.0077, -0.0045, -0.0018,
        -0.0039, -0.0023,  0.0009],
       [-0.0056, -0.0044, -0.0044, -0.0044, -0.0028, -0.0013, -0.0023,
        -0.0007,  0.0027,  0.0005],
       [-0.013 , -0.0137, -0.0072, -0.0032, -0.0018, -0.0022, -0.003 ,
         0.0001,  0.0031, -0.0028],
       [-0.0108, -0.0101, -0.0144, -0.009 , -0.0042, -0.0016, -0.0058,
        -0.0079, -0.0035, -0.0069],
       [-0.0078, -0.0051, -0.0088, -0.0113, -0.0075, -0.003 , -0.0046,
        -0.0028, -0.0068, -0.0051],
       [-0.0083, -0.0041, -0.0047, -0

It worked! There you have it, you can use a COG as a shard in a zarr array. Here we didn't use any compression, but we could have compressed the COG as long as we had used a zarr-compatible codec.

### Is this still a valid COG?

TIFF allows arbitrary bytes within a file, and transitively COG doesn't really care, either. As long as we don't push our tag data too far back from the beginning of the file (such that it can't be retrieved in a single small read), COG really allows us to do most anything we want with extra bytes. But we can also use rio-cogeo to confirm this is true by validating the file.

In [40]:
cog_validate(COG_SHARD)[0]

True

Indeed it validates as expected.

For further confirmation we can open the file with rasterio and see what the pixel values look like. Note that [rasterio doesn't apply the scale or offset](https://github.com/opendatacube/odc-stac/issues/55) for us, so we'll have to do that ourselves.

In [47]:
rds = rasterio.open(COG_SHARD)
(rds.read(1) * 0.0001 - 0.1)[1020:1030,1020:1030]

array([[-0.0029,  0.0003, -0.0015, -0.0017, -0.0003, -0.0053, -0.007 ,
        -0.0085, -0.0038, -0.0017],
       [ 0.0026,  0.0019, -0.0013, -0.0041, -0.0043, -0.0066, -0.0073,
        -0.0081, -0.0037, -0.0016],
       [ 0.0036, -0.0009, -0.0037, -0.006 , -0.007 , -0.0043, -0.0018,
        -0.0039, -0.0051, -0.0004],
       [-0.0042, -0.0041, -0.0055, -0.0053, -0.0088, -0.0071, -0.0021,
        -0.0034, -0.003 ,  0.0005],
       [ 0.0025,  0.0026, -0.0013,  0.0004, -0.0077, -0.0045, -0.0018,
        -0.0039, -0.0023,  0.0009],
       [-0.0056, -0.0044, -0.0044, -0.0044, -0.0028, -0.0013, -0.0023,
        -0.0007,  0.0027,  0.0005],
       [-0.013 , -0.0137, -0.0072, -0.0032, -0.0018, -0.0022, -0.003 ,
         0.0001,  0.0031, -0.0028],
       [-0.0108, -0.0101, -0.0144, -0.009 , -0.0042, -0.0016, -0.0058,
        -0.0079, -0.0035, -0.0069],
       [-0.0078, -0.0051, -0.0088, -0.0113, -0.0075, -0.003 , -0.0046,
        -0.0028, -0.0068, -0.0051],
       [-0.0083, -0.0041, -0.0047, -0

And those are the same values we saw before reading the file as a zarr!

## Conclusion

COG vs Zarr is a false dichotomy. COG and Zarr are two sides of the same coin. Instead of pitting the formats against one another, we should be looking at how they can complement one another, how we can leverage the strengths of both. And if people have usability problems with one, it likely affects the other, so as a community we should be working to improve education, tooling, and guidance, to help people better leverage and work with each of these formats.